In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
%pip install spotipy
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import time

Note: you may need to restart the kernel to use updated packages.


In [29]:
grammyAwards = pd.read_csv("data/grammySongs_1999-2019.csv")
grammyAwards = grammyAwards[['Name', 'Artist', 'GrammyAward', 'GrammyYear', 'Genre']]
grammyAwards = grammyAwards.groupby(['Name', 'Artist', 'GrammyYear', 'Genre']).count().reset_index()
grammyAwards = grammyAwards.applymap(lambda s: s.upper() if type(s) == str else s)

# grammyAwards.head(50)

In [14]:
grammyAwards.head()

,Name,Artist,GrammyYear,Genre,GrammyAward
0,'TIL SUMMER COMES AROUND,KEITH URBAN,2010,COUNTRY,1
1,24 FRAMES,"ASON ISBELL, SONGWRITER",2015,AMERICAN ROOTS MUSIC,1
2,24K MAGIC,BRUNO MARS,2017,GENERAL,1
3,500 MILES HIGH,"CHICK COREA, SOLOIST",2011,JAZZ,1
4,99 PROBLEMS,JAY-Z,2004,RAP,1


In [56]:
import json
auth_manager = SpotifyClientCredentials(client_id = '7606efb22a6848bdbf48ecd67484b9d5', 
                                        client_secret = '5febce52b67b499db9f7295c5f6839ec')
sp = spotipy.Spotify(auth_manager=auth_manager)
song = sp.search(q=f"track:CITY OF BLINDING LIGHTS, artist:U2", type="track", limit=2)
print(song['tracks']['items'][0]['album']['release_date'])

2004-01-01


In [31]:
for index, row in grammyAwards.iterrows():
    artistName = row['Artist']
    trackName = row['Name']
    year = row['GrammyYear']
    try: 
        artistList = artistName.split(",")
        artistName = artistList[0]
        artistName = artistName.split("&")[0]
        artistName = artistName.split(":")[0]
        artistName = artistName.split(";")[0]
        artistName = artistName.split(":")[0]
        artistName = artistName.replace("-", " ")
        if 'FEATURING' in artistName:
            grammyAwards.at[index, 'Artist'] = artistName.split('FEATURING')[0]
            continue
        if 'SOLOIST' in row['Artist']:
            grammyAwards.at[index, 'Artist'] = artistName.split(",")[0]
            continue
        if (artistName != row['Artist']):
            song = sp.search(q=f"track:{trackName}, year:{year}", type="track", limit=2)
            if len(song['tracks']['items']) == 0:
                song = sp.search(q=f"track:{trackName}", type="track", limit=2)
            artist = song['tracks']['items'][0]['artists'][0]['name'].upper()
    #         artistID = song['tracks']['items'][0]['artists'][0]['uri']
    #         artistGenres = sp.artist(artistID)['genres']
    #         if len(artistGenres) == 0:
    #             print(trackName, artistID)
    #         print(artistGenres)
            grammyAwards.at[index, 'Artist'] = artist
    except:
        print("track not found", trackName)


track not found A FEATHER'S NOT A BIRD


In [16]:
grammyAwards.head(50)
grammys_with_genre = grammyAwards

In [ ]:
grammyAwards = grammyAwards[['Name', 'Artist', 'GrammyAward', 'GrammyYear']]
grammyAwards = grammyAwards.groupby(['Name', 'Artist', 'GrammyYear']).count().reset_index()
grammyAwards['GrammyAward']

In [ ]:
grammys_with_genre

In [60]:
# RUN THIS
grammyAwards['Artist'] = grammyAwards['Artist'].apply(lambda s: s.replace("-", " "))
grammyAwards = grammyAwards.drop_duplicates(subset=['Name', 'Artist'], keep='last').reset_index(drop=True)
# grammyAwards.head(50)

In [61]:
grammyAwards

,Name,Artist,Date,Genre,GrammyAward
0,'TIL SUMMER COMES AROUND,KEITH URBAN,2010,COUNTRY,1
1,24 FRAMES,JASON ISBELL,2015,AMERICAN ROOTS MUSIC,1
2,24K MAGIC,BRUNO MARS,2017,GENERAL,1
3,500 MILES HIGH,CHICK COREA,2011,JAZZ,1
4,99 PROBLEMS,JAY Z,2004,RAP,1
...,...,...,...,...,...
356,YOU DON'T KNOW MY NAME,ALICIA KEYS,2004,R&B,1
357,YOU GOT ME,THE ROOTS,1999,RAP,1
358,YOU SAY,LAUREN DAIGLE,2018,GOSPEL/CONTEMPORARY CHRISTIAN MUSIC,1
359,YOU WANT IT DARKER,LEONARD COHEN,2017,ROCK,1


In [66]:
# RUN THIS
# for each song, get the release date, parse to datetime, get the year, set the cell to the year. 
for index, row in grammyAwards.iterrows():
    artistName = row['Artist']
    trackName = row['Name']
    year = row['Date']
    try: 
        song = sp.search(q=f"track:{trackName}, year:{year}", type="track", limit=2)
        if len(song['tracks']['items']) == 0:
            song = sp.search(q=f"track:{trackName}", type="track", limit=2)
        release_date = song['tracks']['items'][0]['album']['release_date']
        release_year = pd.to_datetime(release_date).year
        grammyAwards.at[index, 'Date'] = release_year
    except:
        print("track not found", trackName)
grammyAwards
# song['tracks']['items'][0]['album']['release_date']

track not found NOT AFRAID
track not found NOT READY TO MAKE NICE
track not found STAY WITH ME (DARKCHILD VERSION)
track not found A FEATHER'S NOT A BIRD


,Name,Artist,Date,Genre,GrammyAward
0,'TIL SUMMER COMES AROUND,KEITH URBAN,2010,COUNTRY,1
1,24 FRAMES,JASON ISBELL,2015,AMERICAN ROOTS MUSIC,1
2,24K MAGIC,BRUNO MARS,2017,GENERAL,1
3,500 MILES HIGH,CHICK COREA,2011,JAZZ,1
4,99 PROBLEMS,JAY Z,2004,RAP,1
...,...,...,...,...,...
356,YOU DON'T KNOW MY NAME,ALICIA KEYS,2004,R&B,1
357,YOU GOT ME,THE ROOTS,1999,RAP,1
358,YOU SAY,LAUREN DAIGLE,2018,GOSPEL/CONTEMPORARY CHRISTIAN MUSIC,1
359,YOU WANT IT DARKER,LEONARD COHEN,2017,ROCK,1


In [18]:
billboardHot100 = pd.read_csv('data/billboardHot100_1999-2019.csv')
billboardHot100 = billboardHot100[['Artists', 'Name', 'Genre', 'Peak.position', 'Weeks.on.chart','Weekly.rank','Date']]
billboardHot100['Weeks.on.chart'] = billboardHot100['Weeks.on.chart'].fillna(1)
billboardHot100['Peak.position'] = billboardHot100['Peak.position'].fillna(100)
billboardHot100['Weekly.rank'] = billboardHot100['Weekly.rank'].fillna(100)
billboardHot100['Genre'] = billboardHot100['Genre'].fillna('None')
billboardHot100['Date'] = billboardHot100['Date'].fillna('1999')
billboardHot100 = billboardHot100.groupby(['Artists', 'Name','Genre']).agg({'Peak.position':'min', 'Weeks.on.chart':'max', 'Weekly.rank':'mean', 'Date':'max'})[['Peak.position','Weeks.on.chart','Weekly.rank', 'Date']].reset_index()
billboardHot100.rename(columns = {'Weekly.rank':'Avg.rank'}, inplace = True)
billboardHot100.rename(
    columns={"Artists":"Artist"}, inplace=True)
billboardHot100['Date'] = pd.to_datetime(billboardHot100['Date'])
billboardHot100['Date'] = billboardHot100['Date'].apply(lambda x : x.year)
billboardHot100 = billboardHot100.applymap(lambda s: s.upper() if type(s) == str else s)
billboardHot100.head()

,Artist,Name,Genre,Peak.position,Weeks.on.chart,Avg.rank,Date
0,"""WEIRD AL"" YANKOVIC",CANADIAN IDIOT,"COMEDY,PARODY,ROCK",82.0,3.0,91.333333,2006
1,"""WEIRD AL"" YANKOVIC",WORD CRIMES,"PARODY,COMEDY,POP",39.0,2.0,65.000000,2014
2,'N SYNC,BYE BYE BYE,"ADULT CONTEMPORARY,DANCE-POP,TEEN POP,BOY BAND...",4.0,23.0,14.260870,2000
3,'N SYNC,GONE,"ADULT CONTEMPORARY,ACOUSTIC,SOUL POP,R&;B,BOY ...",11.0,24.0,27.125000,2001
4,'N SYNC,IT'S GONNA BE ME,"MEMES,DANCE-POP,TEEN POP,BOY BAND,POP",1.0,25.0,21.680000,2000


In [19]:
for index, row in billboardHot100.iterrows():
    artistName = row['Artist']
    trackName = row['Name']
    year = row['Date']
    try: 
        song = sp.search(q=f"track:{trackName}, year:{year}", type="track", limit=2)
        if len(song['tracks']['items']) == 0:
            song = sp.search(q=f"track:{trackName}", type="track", limit=2)
#         artist = song['tracks']['items'][0]['artists'][0]['name']
#         billboardHot100.at[index, 'Artist'] = artist
        track_id = song['tracks']['items'][0]['uri']
        song_info = sp.audio_features(track_id)           
        billboardHot100.at[index, 'Energy'] = song_info[0].get('energy')
        billboardHot100.at[index, 'Danceability'] = song_info[0].get('danceability')
        billboardHot100.at[index, 'Tempo'] = song_info[0].get('tempo')
        billboardHot100.at[index, 'Valence'] = song_info[0].get('valence')
    except:
        print("track not found", trackName)
    if (index % 500 == 0):
        print(index)
billboardHot100.to_csv("billboard_features.csv")

0
track not found CAN'T LEAVE WITHOUT IT
track not found THAT'D BE ALRIGHT
track not found DOESN'T REMIND ME
track not found DON'T SAY NO, JUST SAY YES
track not found NOT READY TO DIE
track not found NOT FOR LONG
500
track not found I'LL JUST HOLD ON
track not found DON'T ASK ME HOW I KNOW
track not found YOU'LL ALWAYS BE LOVED BY ME
track not found DON'T GET TOO HIGH
track not found SORRY NOT SORRY
1000
track not found NOT MYSELF TONIGHT
track not found AND I
track not found COULDN'T LAST A MOMENT
track not found WHAT'CHU LIKE
1500
track not found SORRY NOT SORRY
track not found NOT READY TO MAKE NICE
track not found CAN'T HAVE EVERYTHING
track not found HE LOVES U NOT
2000
track not found NOT AFRAID
track not found NOT ALIKE
track not found NOT OVER YOU
track not found A HOUSE IS NOT A HOME
track not found AND I AM TELLING YOU I'M NOT GOING
track not found DON'T STAND SO CLOSE TO ME / YOUNG GIRL
2500
track not found ONE LESS BELL TO ANSWER / A HOUSE IS NOT A HOME
track not found NOT

In [21]:
# billboardHot100.head(50)
print(grammyAwards[grammyAwards['Artist'] == 'BRUNO MARS'])

                     Name      Artist  GrammyYear    Genre  GrammyAward
2               24K MAGIC  BRUNO MARS        2017  GENERAL            1
162  JUST THE WAY YOU ARE  BRUNO MARS        2010      POP            1
285    THAT'S WHAT I LIKE  BRUNO MARS        2017      R&B            2


In [24]:
billboardHot100

,Artist,Name,Genre,Peak.position,Weeks.on.chart,Avg.rank,Date,Energy,Danceability,Tempo,Valence
0,"""WEIRD AL"" YANKOVIC",CANADIAN IDIOT,"COMEDY,PARODY,ROCK",82.0,3.0,91.333333,2006,0.697,0.543,185.978,0.861
1,"""WEIRD AL"" YANKOVIC",WORD CRIMES,"PARODY,COMEDY,POP",39.0,2.0,65.000000,2014,0.430,0.897,121.987,0.964
2,'N SYNC,BYE BYE BYE,"ADULT CONTEMPORARY,DANCE-POP,TEEN POP,BOY BAND...",4.0,23.0,14.260870,2000,0.926,0.610,172.638,0.861
3,'N SYNC,GONE,"ADULT CONTEMPORARY,ACOUSTIC,SOUL POP,R&;B,BOY ...",11.0,24.0,27.125000,2001,0.409,0.704,113.863,0.495
4,'N SYNC,IT'S GONNA BE ME,"MEMES,DANCE-POP,TEEN POP,BOY BAND,POP",1.0,25.0,21.680000,2000,0.873,0.636,165.071,0.908
...,...,...,...,...,...,...,...,...,...,...,...
7209,WILL.I.AM,FALL DOWN,RAP,100.0,1.0,58.000000,2013,0.902,0.538,159.925,0.723
7210,WILL.I.AM,I GOT IT FROM MY MAMA,RAP,31.0,9.0,56.666667,2007,0.777,0.888,118.998,0.876
7211,WILL.I.AM,IT'S A NEW DAY,RAP,78.0,2.0,81.000000,2008,0.644,0.759,119.804,0.459
7212,"WILL.I.AM, BRITNEY SPEARS",SCREAM & SHOUT,RAP,3.0,24.0,14.583333,2012,0.686,0.797,127.990,0.808


In [28]:
print(billboardHot100[billboardHot100['Artist'] == ''])

Empty DataFrame
Columns: [Artist, Name, Genre, Peak.position, Weeks.on.chart, Avg.rank, Date, Energy, Danceability, Tempo, Valence]
Index: []


In [34]:
grammyAwards

,Name,Artist,GrammyYear,Genre,GrammyAward
0,'TIL SUMMER COMES AROUND,KEITH URBAN,2010,COUNTRY,1
1,24 FRAMES,JASON ISBELL,2015,AMERICAN ROOTS MUSIC,1
2,24K MAGIC,BRUNO MARS,2017,GENERAL,1
3,500 MILES HIGH,CHICK COREA,2011,JAZZ,1
4,99 PROBLEMS,JAY-Z,2004,RAP,1
...,...,...,...,...,...
380,YOU DON'T KNOW MY NAME,ALICIA KEYS,2004,R&B,1
381,YOU GOT ME,THE ROOTS,1999,RAP,1
382,YOU SAY,LAUREN DAIGLE,2018,GOSPEL/CONTEMPORARY CHRISTIAN MUSIC,1
383,YOU WANT IT DARKER,LEONARD COHEN,2017,ROCK,1


In [67]:
# grammyAwards[]
grammyAwards.rename(columns={"GrammyYear":"Date"}, inplace=True)
combined_df = grammyAwards.merge(billboardHot100, on=['Name', 'Date'], how='left')
combined_df.head(50)

,Name,Artist_x,Date,Genre_x,GrammyAward,Artist_y,Genre_y,Peak.position,Weeks.on.chart,Avg.rank,Energy,Danceability,Tempo,Valence
0,'TIL SUMMER COMES AROUND,KEITH URBAN,2010,COUNTRY,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24 FRAMES,JASON ISBELL,2015,AMERICAN ROOTS MUSIC,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24K MAGIC,BRUNO MARS,2017,GENERAL,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,500 MILES HIGH,CHICK COREA,2011,JAZZ,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,99 PROBLEMS,JAY Z,2004,RAP,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ADORN,MIGUEL,2012,R&B,1,MIGUEL,"POP,NEO SOUL,SINGER-SONGWRITER,TRIP-HOP,SOUL,S...",17.0,33.0,36.060606,0.576,0.625,179.063,0.235
6,AGAIN,LENNY KRAVITZ,2000,ROCK,1,LENNY KRAVITZ,"SOUL,PRODUCER,SINGER-SONGWRITER,ALTERNATIVE RO...",4.0,32.0,18.750000,0.834,0.751,95.053,0.894
7,AIN'T NO OTHER MAN,CHRISTINA AGUILERA,2012,POP,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AIN'T IT FUN,PARAMORE,2014,ROCK,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ALRIGHT,KENDRICK LAMAR,2015,RAP,1,KENDRICK LAMAR,"HIP-HOP,POP,POLITICS,SCREEN,PRODUCER,POST-BOP,...",81.0,14.0,88.000000,0.766,0.796,110.034,0.558


In [70]:
%pip install python-levenshtein

     |████████████████████████████████| 50 kB 3.9 MB/s eta 0:00:011
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp38-cp38-macosx_10_9_x86_64.whl size=81452 sha256=1a2d5c09fb174bd4d51039f9651d7baf11d98ab80c6ae6cd9192e0b6a33bd3f1
  Stored in directory: /Users/neilxu/Library/Caches/pip/wheels/d7/0c/76/042b46eb0df65c3ccd0338f791210c55ab79d209bcc269e2c7
Successfully built python-levenshtein
Note: you may need to restart the kernel to use updated packages.


In [83]:
'''
for each one, search in the billboard for the artistName, true if satisfies fuzzy and 
'''
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

for index, row in grammyAwards.iterrows():
    if index % 50 == 0:
        print(index)
    artist1 = row['Artist']
    track = row['Name']
    found = False
    for id2, r2 in billboardHot100.iterrows():
        artist2 = r2['Artist']
        t2 = r2['Name']
        artistRatio = fuzz.partial_ratio(artist1, artist2)
        trackRatio = fuzz.partial_ratio(track, t2)
        if trackRatio > 90 and artistRatio > 90:
            print(artist1, artist2)
            print(track, t2)
            found = True
            grammyAwards.at[index, 'Energy'] = r2['Energy']
            grammyAwards.at[index, 'Danceability'] = r2['Danceability']
            grammyAwards.at[index, 'Tempo'] = r2['Tempo']
            grammyAwards.at[index, 'Valence'] = r2['Valence']
            grammyAwards.at[index, 'Peak.position'] = r2['Peak.position']
            grammyAwards.at[index, 'Weeks.on.chart'] = r2['Weeks.on.chart']
            grammyAwards.at[index, 'Avg.rank'] = r2['Avg.rank']
            break
    if not found:
        grammyAwards.at[index, 'Energy'] = -1
        grammyAwards.at[index, 'Danceability'] = -1
        grammyAwards.at[index, 'Tempo'] = -1
        grammyAwards.at[index, 'Valence'] = -1
        grammyAwards.at[index, 'Peak.position'] = -1
        grammyAwards.at[index, 'Weeks.on.chart'] = -1
        grammyAwards.at[index, 'Avg.rank'] = -1
        print('no match in top 100', track, artist1)
grammyAwards.head(10)
#Peak.position	Weeks.on.chart	Avg.rank

0
no match in top 100 'TIL SUMMER COMES AROUND KEITH URBAN
no match in top 100 24 FRAMES JASON ISBELL
BRUNO MARS BRUNO MARS
24K MAGIC 24K MAGIC
no match in top 100 500 MILES HIGH CHICK COREA
JAY Z JAY
99 PROBLEMS 99 PROBLEMS
MIGUEL MIGUEL
ADORN ADORN
LENNY KRAVITZ LENNY KRAVITZ
AGAIN AGAIN
CHRISTINA AGUILERA CHRISTINA AGUILERA
AIN'T NO OTHER MAN AIN'T NO OTHER MAN
PARAMORE PARAMORE
AIN'T IT FUN AIN'T IT FUN
KENDRICK LAMAR KENDRICK LAMAR
ALRIGHT ALRIGHT
LENNY KRAVITZ LENNY KRAVITZ
AMERICAN WOMAN AMERICAN WOMAN
no match in top 100 ANGEL LALAH HATHAWAY
no match in top 100 ANGRY WORLD NEIL YOUNG
SYSTEM OF A DOWN SYSTEM OF A DOWN
B.Y.O.B. B.Y.O.B.
LADY GAGA LADY GAGA
BAD ROMANCE BAD ROMANCE
SKRILLEX  SKRILLEX
BANGARANG BANGARANG
no match in top 100 BARTON HOLLOW THE CIVIL WARS
no match in top 100 BE OK WILLI.AM
MARY J BLIGE MARY J. BLIGE
BE WITHOUT YOU BE WITHOUT YOU
CHRISTINA AGUILERA CHRISTINA AGUILERA
BEAUTIFUL BEAUTIFUL
U2 U2
BEAUTIFUL DAY BEAUTIFUL DAY
CARRIE UNDERWOOD CARRIE UNDERWOOD

BRUNO MARS BRUNO MARS
JUST THE WAY YOU ARE JUST THE WAY YOU ARE
no match in top 100 KEEP ON THE SUNNY SIDE JUNE CARTER CASH
no match in top 100 KID SISTER THE TIME JUMPERS
no match in top 100 KILLER DILLER BLUES ALABAMA SHAKES
CHRISTINA AGUILERA CHRISTINA AGUILERA, LIL' KIM, MYA, P!NK
LADY MARMALADE LADY MARMALADE
no match in top 100 LAKE BY THE OCEAN MAXWELL
CARRIE UNDERWOOD CARRIE UNDERWOOD
LAST NAME LAST NAME
no match in top 100 LAZARETTO JACK WHITE
IDINA MENZEL IDINA MENZEL
LET IT GO LET IT GO
THE BLACK EYED PEAS THE BLACK EYED PEAS
LET'S GET IT STARTED LET'S GET IT STARTED
BRAD PAISLEY BRAD PAISLEY
LETTER TO ME LETTER TO ME
no match in top 100 LITTLE GHETTO BOY LALAH HATHAWAY
no match in top 100 LITTLE THINGS INDIAARIE
TIM MCGRAW TIM MCGRAW
LIVE LIKE YOU WERE DYING LIVE LIKE YOU WERE DYING
LIL WAYNE  LIL WAYNE
LOLLIPOP LOLLIPOP
THE BLACK KEYS THE BLACK KEYS
LONELY BOY LONELY BOY
EMINEM EMINEM
LOSE YOURSELF LOSE YOURSELF
no match in top 100 LOVE OF MY LIFE (AN ODE TO HIP HOP) ERYKA

no match in top 100 WHAT GOES AROUND...COMES AROUND JUSTIN TIMBERLAKE
no match in top 100 WHAT A BEAUTIFUL NAME HILLSONG WORSHIP
no match in top 100 WHAT YOU KNOW TI.
no match in top 100 WHENEVER I SAY YOUR NAME STING
no match in top 100 WHIPLASH MOTÖRHEAD
no match in top 100 WHISKEY IN THE JAR METALLICA
TAYLOR SWIFT TAYLOR SWIFT
WHITE HORSE WHITE HORSE
BAHA MEN BAHA MEN
WHO LET THE DOGS OUT WHO LET THE DOGS OUT
CREED CREED
WITH ARMS WIDE OPEN WITH ARMS WIDE OPEN
no match in top 100 WOMAN WOLFMOTHER
no match in top 100 WORK IT MISSY ELLIOTT
KEITH URBAN KEITH URBAN
YOU'LL THINK OF ME YOU'LL THINK OF ME
no match in top 100 A CHANGE IS GONNA COME HERBIE HANCOCK
no match in top 100 A FEATHER'S NOT A BIRD ROSANNE CASH, ARTIST
LIL WAYNE LIL WAYNE
A MILLI A MILLI
FOO FIGHTERS FOO FIGHTERS
ALL MY LIFE ALL MY LIFE
JANET JACKSON JANET
ALL FOR YOU ALL FOR YOU
no match in top 100 ALL OF THE LIGHTS EVIL EMPIRE
KENDRICK LAMAR C
I BOYFRIEND/GIRLFRIEND
THE BLACK EYED PEAS THE BLACK EYED PEAS
I GOTTA F

,Name,Artist,Date,Genre,GrammyAward,Energy,Danceability,Tempo,Valence,Peak.position,Weeks.on.chart,Avg.rank
0,'TIL SUMMER COMES AROUND,KEITH URBAN,2010,COUNTRY,1,-1.000,-1.000,-1.000,-1.000,-1.0,-1.0,-1.000000
1,24 FRAMES,JASON ISBELL,2015,AMERICAN ROOTS MUSIC,1,-1.000,-1.000,-1.000,-1.000,-1.0,-1.0,-1.000000
2,24K MAGIC,BRUNO MARS,2017,GENERAL,1,0.803,0.818,106.970,0.632,4.0,41.0,21.439024
3,500 MILES HIGH,CHICK COREA,2011,JAZZ,1,-1.000,-1.000,-1.000,-1.000,-1.0,-1.0,-1.000000
4,99 PROBLEMS,JAY Z,2004,RAP,1,0.887,0.494,89.554,0.548,30.0,12.0,61.583333
5,ADORN,MIGUEL,2012,R&B,1,0.576,0.625,179.063,0.235,17.0,33.0,36.060606
6,AGAIN,LENNY KRAVITZ,2000,ROCK,1,0.834,0.751,95.053,0.894,4.0,32.0,18.750000
7,AIN'T NO OTHER MAN,CHRISTINA AGUILERA,2012,POP,1,0.898,0.681,127.966,0.626,6.0,20.0,19.555556
8,AIN'T IT FUN,PARAMORE,2014,ROCK,1,0.815,0.534,103.911,0.670,10.0,24.0,29.958333
9,ALRIGHT,KENDRICK LAMAR,2015,RAP,1,0.766,0.796,110.034,0.558,81.0,14.0,88.000000


In [84]:
grammyAwards.to_csv("grammy_award_data.csv")

In [ ]:
bruno_only = billboardHot100[billboardHot100['Artist'] == 'BRUNO MARS']
# print(bruno_only)
for index, row in bruno_only.iterrows():
    artistName = row['Artist']
    trackName = row['Name']
    year = row['Date']
    try: 
        song = sp.search(q=f"track:{trackName}, year:{year}", type="track", limit=2)
        
        if len(song['tracks']['items']) == 0:
            song = sp.search(q=f"track:{trackName}", type="track", limit=2)
            
#         artist = song['tracks']['items'][0]['artists'][0]['name']
#         bruno_only.at[index, 'Artist'] = artist
        track_id = song['tracks']['items'][0]['uri']
        song_info = sp.audio_features(track_id)           
        bruno_only.at[index, 'Energy'] = song_info[0].get('energy')
        bruno_only.at[index, 'Danceability'] = song_info[0].get('danceability')
        bruno_only.at[index, 'Tempo'] = song_info[0].get('tempo')
        bruno_only.at[index, 'Valence'] = song_info[0].get('valence')
    except:
        print("track not found", trackName)
    if (index % 500 == 0):
        print(index)

In [ ]:
# bruno_only.head(50)

In [ ]:
bruno_grammy = grammyAwards.merge(bruno_only.head(100), on=['Name', 'Artist'], how='left')
bruno_grammy.head(50)

In [ ]:
# kendrick_only = billboardHot100[billboardHot100['Artist'] == 'KENDRICK LAMAR']
# # print(bruno_only)
# for index, row in kendrick_only.iterrows():
#     artistName = row['Artist']
#     trackName = row['Name']
#     year = row['Date']
#     try: 
#         song = sp.search(q=f"track:{trackName}, year:{year}", type="track", limit=2)
        
#         if len(song['tracks']['items']) == 0:
#             song = sp.search(q=f"track:{trackName}", type="track", limit=2)
            
# #         artist = song['tracks']['items'][0]['artists'][0]['name']
# #         bruno_only.at[index, 'Artist'] = artist
#         track_id = song['tracks']['items'][0]['uri']
#         song_info = sp.audio_features(track_id)           
#         kendrick_only.at[index, 'Energy'] = song_info[0].get('energy')
#         kendrick_only.at[index, 'Danceability'] = song_info[0].get('danceability')
#         kendrick_only.at[index, 'Tempo'] = song_info[0].get('tempo')
#         kendrick_only.at[index, 'Valence'] = song_info[0].get('valence')
#     except:
#         print("track not found", trackName)
#     if (index % 500 == 0):
#         print(index)

In [ ]:
kendrick_grammy = grammyAwards.merge(kendrick_only.head(100), on=['Name', 'Artist'], how='left')
# kendrick_grammy.head(50)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

genres_as_list = [g.split(",") for g in billboardHot100['Genre']]
grammy_genres
one_hot = MultiLabelBinarizer()
genre_encoding = one_hot.fit_transform(genres_as_list)
genres = one_hot.classes_

genre_df = pd.DataFrame(genre_encoding, columns = genres)
# genre_df.head()
billboardHot100 = billboardHot100.join(genre_df)
billboardHot100 = billboardHot100.drop(columns=['Genre'])
billboardHot100

In [ ]:
import matplotlib.pyplot as plt
# print(genres)
genre_count = []
genre_dict= {}
for genre in genres:
    count = billboardHot100[genre].sum()
    genre_count.append(count)
    genre_dict[genre] = count
# fig = plt.figure()
print(genre_dict)
plt.bar(genres, genre_count)
plt.show()

In [ ]:
sorted_genres = dict(sorted(genre_dict.items(), key=lambda genre:genre[1]))
print(sorted_genres)

In [ ]:
# list(sorted_genres.keys())
popular_genres_top_50 = list(sorted_genres.keys())[-50:]

In [ ]:
billboardHot100[list(billboardHot100.columns[:6]) + popular_genres_top_50]